# The Autonomous Vehicle Co-pilot
Here is a chatbot, empowered by GPT-3.5, running to help the vehicle driver. 

He is set as a co-pilot, analyzing the environment based on other multi-modality input, which has been processed as text so he can reason and identify potential risk. 


## Setup
Notice: if you tried to run this program on your linux server, 
and you are in China,
make sure that you opened the vpn. 
Otherwise the api account may be forbiddened!

In [7]:
import os
import openai

def read_key_from_file(filepath):
    with open(filepath, 'r') as file:
        return file.read().strip()
personal_key = read_key_from_file('apikey.cn')
organ = read_key_from_file('organization.cn')

openai.organization = organ
openai.api_key  = personal_key

TEMPERATURE = 0

In [12]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature= TEMPERATURE, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo"):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=TEMPERATURE, # this is the degree of randomness of the model's output
    )
    print(str(response.choices[0].message))
    return response.choices[0].message["content"]

In [11]:
blind_guide = """ We are currently driving on a city street filled with green plants, with bicycle lanes on both sides of the road. On the left bicycle lane, there are three cyclists moving slowly, while on the right side, there are two cyclists riding at a faster pace.
Next, there is a spacious pedestrian walkway, with five pedestrians walking on the left side and four pedestrians on the right side, two of whom are jogging. Each side of the street is lined with banyan trees, providing ample shade for pedestrians to cool off.
We are heading towards an intersection. The traffic signal at the intersection is displaying a green light, indicating that we can proceed. Each direction at the intersection has only one lane, clearly marked. In front of us, there is a blue sedan driving at a moderate speed."""
chat_history = [
    {'role':'system', 'content':'You are a co-pilot AI assistant designed to help drivers understand their surroundings.'},
    
    {'role':'user', 'content':'What is our current situation on the road?'},
    { 'role':'system','content': blind_guide}
]


another = [
    {'role':'assistant', 'content':'We are currently driving on a city street adorned with greenery, with bicycle lanes present on both sides of the road. On our left, three cyclists are moving at a slow pace, while on the right, two cyclists are moving more quickly.'},

    {'role':'user', 'content':'What about pedestrians and the environment around us?'},

    {'role':'assistant', 'content':'Indeed, there is a spacious pedestrian walkway. On the left side, five pedestrians are walking and on the right side, four pedestrians, two of whom are jogging. The street is lined with banyan trees on either side, providing a shaded canopy for pedestrians.'},

    {'role':'user', 'content':'What lies ahead of us?'},

    {'role':'assistant', 'content':'We are heading towards an intersection. The traffic signal at the intersection currently displays a green light, indicating that we can proceed. Each direction at the intersection has a single lane, clearly marked. A blue sedan is driving at a moderate speed ahead of us.'}
]


# Data Integration
 Firstly, you'll need a way to convert the input from the vehicle's sensors (like LIDAR, RADAR, cameras, etc.) into a format that the chatbot can understand. This might involve using object detection algorithms to identify objects and their positions, then converting this data into a text description of the scene.


## buggy chatbot here
I need to find a good programming way of coding here

In [13]:
def chat_with_assistant(user_input, context):
    # Append the user input to the context
    context.append({'role':'user', 'content':f"{user_input}"})
    
    # Get the assistant's response
    response = get_completion_from_messages(context)
    
    # Append the assistant's response to the context
    context.append({'role':'assistant', 'content':f"{response}"})
    
    return context


# Chat with the assistant
context = chat_with_assistant('Hello, assistant!', chat_history)
print(context)

# Continue the conversation
context = chat_with_assistant('Tell me a joke', context)
print(context)


{
  "role": "assistant",
  "content": "Hello! How can I assist you today?"
}
[{'role': 'system', 'content': 'You are a co-pilot AI assistant designed to help drivers understand their surroundings.'}, {'role': 'user', 'content': 'What is our current situation on the road?'}, {'role': 'system', 'content': ' We are currently driving on a city street filled with green plants, with bicycle lanes on both sides of the road. On the left bicycle lane, there are three cyclists moving slowly, while on the right side, there are two cyclists riding at a faster pace.\nNext, there is a spacious pedestrian walkway, with five pedestrians walking on the left side and four pedestrians on the right side, two of whom are jogging. Each side of the street is lined with banyan trees, providing ample shade for pedestrians to cool off.\nWe are heading towards an intersection. The traffic signal at the intersection is displaying a green light, indicating that we can proceed. Each direction at the intersection ha

2. **Contextual Understanding:** Your chatbot should be capable of maintaining and updating a model of the car's surroundings based on this information. For example, it should keep track of the positions of other vehicles, pedestrians, and road features, updating these as new data comes in.

3. **Real-Time Interaction:** The chatbot should be capable of generating responses in real-time. Delays could be dangerous in a driving context.

4. **Proactive Alerting:** Your co-pilot chatbot should not just react to the driver's queries but should also proactively provide alerts and advice. For example, if it detects a pedestrian stepping onto the road, it should immediately alert the driver.

5. **Simulated Training:** Consider using simulated environments to train and test your chatbot before deploying it in a real vehicle. This can help you ensure that the chatbot behaves correctly in a wide range of situations.

6. **Safety Precautions:** Since the bot is serving in a co-pilot role, it should prioritize safety and assume the driver may be unaware of potential dangers. It should also be prepared to handle ambiguous situations by advising caution.

7. **User Customization:** Different drivers may have different preferences for how much information they want to receive, what kind of language the co-pilot uses, etc. Consider allowing users to customize the chatbot's behavior to some extent.

8. **Integration with Vehicle's Systems:** If possible, the chatbot could also be integrated with the vehicle's systems to provide even more functionality. For example, it could adjust the vehicle's speed or even take control in an emergency. However, this would require a high level of reliability and many legal and ethical considerations.